<a href="https://colab.research.google.com/github/Elzoghost/Classification_des_critiques_de_films/blob/main/IMDB_class_binary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import imdb
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score

In [ ]:
# Chargement des données
max_features = 10000
max_len = 9999
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
x_train = sequence.pad_sequences(x_train, maxlen=max_len)
x_test = sequence.pad_sequences(x_test, maxlen=max_len)

In [ ]:
# Création du modèle
model = Sequential()
model.add(Embedding(max_features, 128, input_length=max_len))
model.add(LSTM(128))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
# Compilation du modèle
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2)

In [ ]:
# Entraînement du modèle
history = model.fit(x_train, y_train, epochs=10, batch_size=64, validation_split=0.2,
                    callbacks=[early_stopping, model_checkpoint, reduce_lr])

In [ ]:
# Chargement du meilleur modèle
model.load_weights('best_model.h5')

In [ ]:
# Évaluation du modèle
loss, accuracy = model.evaluate(x_test, y_test)
y_pred = (model.predict(x_test) >= 0.5).astype(int)
f1 = f1_score(y_test, y_pred)
print(f'Perte (Loss): {loss}, Précision (Accuracy): {accuracy}, F1-Score: {f1}')

In [ ]:
# Matrice de confusion et rapport de classification
confusion = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred, target_names=['Negatif', 'Positif'])

print("Matrice de confusion:\n", confusion)
print("\nRapport de classification:\n", classification_rep)

In [ ]:
from keras.models import load_model
loaded_model = load_model('best_model.h5')


In [ ]:
# Remplacez 'new_review_text' par la nouvelle critique de film que vous souhaitez classifier.
new_review_text = "This movie was not  amazing! I do not loved every moment of it."

# Prétraitement de la nouvelle critique de film (remplacez cette étape par le prétraitement utilisé lors de l'entraînement).
new_review = [imdb.get_word_index().get(word.lower(), 0) for word in new_review_text.split()]
new_review = sequence.pad_sequences([new_review], maxlen=max_len)

# Prédiction avec le modèle chargé.
prediction = loaded_model.predict(new_review)

# Interprétation de la prédiction (par exemple, si la valeur prédite est >= 0.5, considérez la critique comme positive).
if prediction >= 0.5:
    print("Critique positive")
else:
    print("Critique négative")

In [ ]:
# Tracer les graphes de la perte et de la précision
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Perte (Loss)')
plt.title('Évolution de la perte pendant l\'entraînement')
plt.xlabel('Époques')
plt.ylabel('Perte')
plt.grid()

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Précision (Accuracy)')
plt.title('Évolution de la précision pendant l\'entraînement')
plt.xlabel('Époques')
plt.ylabel('Précision')
plt.grid()

plt.show()